In [106]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [110]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51269_2024-05-07~2025-05-07.xlsx')
ka_df = pd.read_excel('/Users/seanyu_mac/Downloads/20250507121140.xlsx')

In [112]:
ka_df.columns

Index(['결제번호', '최초배송비번호', '주문번호', '주문상태', '채널', '발송요청일', '채널상품번호', '상품명', '옵션',
       '수량', '상품금액', '옵션금액', '판매자할인금액', '판매자쿠폰할인금액', '정산기준금액', '기본수수료',
       '노출추가수수료', '추천리워드수수료', '수수료할인금액', '주문일', '송장입력일', '취소완료일', '환불완료일',
       '배송예정일', '판매자상품번호', '배송방법', '택배사', '송장번호', '배송비지불방법', '기본배송비 유형',
       '기본배송비 금액', '도서산간 추가 배송비 금액', '리워드율', '리워드금액', '브랜드', '모델명', '유입경로',
       '옵션코드', '톡딜여부', '상품유형', 'biz판매여부 '],
      dtype='object')

In [114]:
fs_df.columns

Index(['결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정상가',
       '판매가', '쿠폰 할인', '포인트 할인', '적립금', '정산가', '수령인명', '수령인 연락처', '구매자명',
       '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일'],
      dtype='object')

In [116]:
fs_df = fs_df[['결제 번호', '결제 완료일', '주문 상태', '옵션명', '수량', '정산가']]

In [118]:
ka_df = ka_df[['결제번호', '발송요청일', '주문상태', '옵션', '수량', '정산기준금액']]

In [120]:
fs_df['구분'] = '친한스토어'
ka_df['구분'] = '톡스토어'

In [122]:
ka_df['박스수량'] = ka_df['옵션'].apply(lambda x: int(x.split()[1].split('박스')[0]))

In [124]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: int(x.split(' (총')[0].split('박스')[0]))

In [126]:
ka_df.columns = fs_df.columns
df = pd.concat([ka_df, fs_df])
df

,결제 번호,결제 완료일,주문 상태,옵션명,수량,정산가,구분,박스수량
0,2431695200,2025-04-28 23:09:01,204 결제 취소 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
1,2431687218,2025-04-28 22:54:02,204 결제 취소 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
2,2431680091,2025-04-28 22:41:18,305 배송 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
3,2431651764,2025-04-28 21:59:34,305 배송 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
4,2431644406,2025-04-28 21:46:09,305 배송 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
...,...,...,...,...,...,...,...,...
10,parma20250422102510V9gs6,2025-04-22 10:25:42.355000,구매확정,12박스 (총 12개월분)_50%할인,1,260000,친한스토어,12
11,parma202504221025270fWZ7,2025-04-22 10:25:42.029000,구매확정,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6
12,parma20250422102441W5GJ9,2025-04-22 10:25:26.940000,구매확정,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6
13,parma20250422102441YYE44,2025-04-22 10:25:10.354000,구매확정,6박스 (총 6개월분)_45%할인,1,142000,친한스토어,6


In [128]:
df['주문 상태'].unique()

array(['204 결제 취소 완료', '305 배송 완료', '601 구매 결정', '507 반품 결제 취소 완료',
       '구매확정', '취소완료'], dtype=object)

In [130]:
status = ['305 배송 완료', '601 구매 결정', '구매확정']

In [132]:
cancel_df = df[~df['주문 상태'].isin(status)]
cancel_df

,결제 번호,결제 완료일,주문 상태,옵션명,수량,정산가,구분,박스수량
0,2431695200,2025-04-28 23:09:01,204 결제 취소 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
1,2431687218,2025-04-28 22:54:02,204 결제 취소 완료,선택: 3박스 (총 3개월분)_31%할인,1,89000,톡스토어,3
16,2430842008,2025-04-27 22:26:23,204 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
27,2430408997,2025-04-27 10:50:44,204 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
43,2429608584,2025-04-26 00:49:57,204 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
44,2429603101,2025-04-26 00:35:26,204 결제 취소 완료,선택: 12박스 (총 12개월분)_50%할인,1,260000,톡스토어,12
46,2429558761,2025-04-25 23:06:43,204 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
49,2429550675,2025-04-25 22:51:49,204 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
92,2427176537,2025-04-22 23:50:28,507 반품 결제 취소 완료,선택: 6박스 (총 6개월분)_45%할인,1,142000,톡스토어,6
104,2427048262,2025-04-22 20:22:52,204 결제 취소 완료,선택: 12박스 (총 12개월분)_50%할인,1,260000,톡스토어,12


In [134]:
df = df[df['주문 상태'].isin(status)]

In [136]:
df['결제 완료일'] = pd.to_datetime(df['결제 완료일'])
df['결제 완료일'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [138]:
df.rename(columns={'결제 번호': '주문수량'}, inplace=True)
df.rename(columns={'정산가': '매출'}, inplace=True)
result_df = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='합계'))
result_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '옵션명'], values='주문수량', aggfunc='count', fill_value=0, margins=True, margins_name='합계'))
result_df['주문수량'] = result_df1['주문수량']
result_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     옵션명                                           
2025-04-22 12박스 (총 12개월분)_50%할인         780,000   36    3
           3박스 (총 3개월분)_31%할인           178,000    6    2
           6박스 (총 6개월분)_45%할인         1,278,000   54    9
           선택: 12박스 (총 12개월분)_50%할인   1,300,000   60    5
           선택: 3박스 (총 3개월분)_31%할인     1,780,000   60   20
           선택: 6박스 (총 6개월분)_45%할인     2,414,000  102   17
2025-04-23 선택: 12박스 (총 12개월분)_50%할인     780,000   36    3
           선택: 3박스 (총 3개월분)_31%할인       623,000   21    7
           선택: 6박스 (총 6개월분)_45%할인     1,278,000   54    9
2025-04-24 선택: 3박스 (총 3개월분)_31%할인       267,000    9    3
           선택: 6박스 (총 6개월분)_45%할인       710,000   30    5
2025-04-25 선택: 12박스 (총 12개월분)_50%할인     260,000   12    1
           선택: 3박스 (총 3개월분)_31%할인       534,000   18    6
           선택: 6박스 (총 6개월분)_45%할인     1,420,000   60   10
2025-04-26 선택: 12박스 (총 12개월분)_50%할인     260,000   12    1
           선택: 3박스 (총 3개월분)_31%할인       534,000   18    6
           선택: 6박스 (총 6개월분)_45%할인       710,000   30    5
2025-04-27 선택: 12박스 (총 12개월분)_50%할인     260,000   12    1
           선택: 3박스 (총 3개월분)_31%할인       534,000   18    6
           선택: 6박스 (총 6개월분)_45%할인     1,136,000   48    8
2025-04-28 선택: 12박스 (총 12개월분)_50%할인     260,000   12    1
           선택: 3박스 (총 3개월분)_31%할인       445,000   15    5
           선택: 6박스 (총 6개월분)_45%할인       852,000   36    6
합계                                   18,593,000  759  139

In [140]:
result_df = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='합계'))
result_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values='주문수량', aggfunc='count', fill_value=0, margins=True, margins_name='합계'))
result_df['주문수량'] = result_df1['주문수량']
result_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     구분                         
2025-04-22 친한스토어   2,236,000   96   14
           톡스토어    5,494,000  222   42
2025-04-23 톡스토어    2,681,000  111   19
2025-04-24 톡스토어      977,000   39    8
2025-04-25 톡스토어    2,214,000   90   17
2025-04-26 톡스토어    1,504,000   60   12
2025-04-27 톡스토어    1,930,000   78   15
2025-04-28 톡스토어    1,557,000   63   12
합계                18,593,000  759  139